# FHE libraries benchmarking

In [3]:
import time  # Pour mesurer le temps d'exécution

## Concrete

In [1]:
from concrete import fhe
from typing import Final, List
import time
import matplotlib.pyplot as plt

def add(x: int, y: int) -> int:
    return x + y

def multiply(x: int, y: int) -> int:
    return x * y

input_sizes: Final[List[int]] = [5, 10, 50, 100, 500, 1000, 5000, 10000, 50000, 100000] # Varying input sizes in log scale
num_runs: Final[int] = 10  # Number of times to run the test for each input size

encrypt_time: List[int] = []
add_execution_times: List[int] = []
multiply_execution_times: List[int] = []
decrypt_time: List[int] = []

## Measuring encryption times

In [ ]:
encrypt_time = []

for size in input_sizes:
    start_time = time.time()

    
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    encrypt_time.append(elapsed_time)

plt.plot(input_sizes, encrypt_time)
plt.xlabel('Input Size')
plt.ylabel('Encryption Time (s)')
plt.title('Encryption Time vs Input Size')
plt.show()